In [3]:
%load_ext nb_black
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats

%matplotlib inline
set_matplotlib_formats("png")
plt.style.use("seaborn-white")
import numpy as np
import seaborn as sns
from IPython.display import SVG

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

# Lecture 12
## Coalescent Hidden markov models

In the past few weeks, we studied inference methods which assume that evolution at each gene occurs indepenedently. We studied the allele frequency spectrum, which describes the distribution of the number of variant copies of a gene (Lecture 8), as well as the Pritchard-Stephens-Donnelly model (Lecture 10), which clusters together different samples based on a model that assumes complete independence between gene frequencies at different genes. 

This week, we'll go back to thinking about recombination and genes that evolve under the presence of linkage disequilibrium. We will use hidden Markov modeling to perform inference under this set of assumptions.

We already saw an example of a hidden Markov model when we described the sequentially Markov coalescent (Lecture 6). In that lecture, we first generated a sequence of trees for two leaves, and then dropped mutations on them to generate data. 

We will extend this model in two ways:
1. Still for sample size $n=2$, we will infer this sequence of trees, and use it to estimate the size of the population backwards in time.
2. For sample size $n>2$, inferring trees becomes too complicated, so we will make a (strong) simplifying assumption regarding the nature of tree topologies, which leads to a simple yet powerful algorithm for inferring ancestry.

Bullet point #1 leads to the program PSMC ([Li & Durbin; *Nature*, 2011](https://www.nature.com/articles/nature10231)). Bullet point #2 leads to the method of Li & Stephens ([*Genetics*, 2003](https://www.genetics.org/content/165/4/2213.long)). Both of these papers are extremely influential.

## The Li-Stephens haplotype copying model
Li and Stephens (2003) were interested in computing the likelihood of a sample $H_1,\dots,H_n$ of haplotypic data

$$\mathbb{P}_\theta(H_1,\dots,H_n),$$

where $\theta$ represents various parameters of some evolutionary model (in this case, recombination.)

Their insight was that, if we had a good approximation for the *conditional* probability $\tilde{P}_\theta(H_i\mid H_{i-1},\dots,H_1)$, then we could use it to perform likelihood-based inference.

Using the chain rule of probability, we have 
\begin{align}
\mathbb{P}_\theta(H_1,\dots,H_n) &= \pi_\theta(H_1) \prod_{i=2}^n \mathbb{P}_\theta(H_i\mid H_{i-1},\dots,H_1) \\
&\approx \pi_\theta(H_1) \prod_{i=2}^n \tilde{\mathbb{P}}_\theta(H_i\mid H_{i-1},\dots,H_1) \\
&=: \tilde{\mathbb{P}}_\theta(H_1,\dots,H_n).
\end{align}

This is known as the *product of approximation conditionals* (PAC) likelihood.

How should we approach $\tilde{\mathbb{P}}(H_i\mid H_{i-1},\dots,H_1)$?

![li stephens](https://www.genetics.org/content/genetics/165/4/2213/F2.large.jpg)

(graphic from original paper)

We say that each underlying haplotype is an imperfect mosaic of the preceding haplotypes. (Note: for simplicity, I will focus on the case where we have haplotypic data, however their model transfers readily to diploid genotype data.)

Given a haplotype $H_{n+1} = (H_{n+1,1},\dots,H_{n+1,L})$, let $Y_\ell\in{1,\dots,n}$ denote the identity of the copied-from haplotype at position $\ell$. In the picture above, if we focus on $h_{4A}$, then $Y_1=Y_2=3$, $Y_3=Y_4=Y_5=2$.

## Hidden Markov modeling
The LS method is naturally modeled using an hidden Markov model. The "hidden state" is $Y_i$ in the notation above, and the "emission" is the probability of observing $H_{n+1,\ell}$ given that $Y_\ell=h \in \{1,\dots,n\}$.

The likelihood for an HMM is:

$$p(H_{n+1,1}=a_1,\dots,H_{n+1,L}=a_L)=\sum_{h_1,\dots,h_L} \underbrace{p(Y_1=h_1)}_\text{stationary}\prod_{\ell=2}^L \underbrace{p(Y_\ell=h_\ell\mid Y_{\ell-1}=h_{\ell-1})}_\text{transition} \prod_{i=1}^L p\underbrace{(H_{n+1,\ell}=a_\ell \mid Y_{\ell}=h_\ell)}_\text{emission}.$$

Note that the sum has exponentially terms, but we will use the Markov structure to evaluate it in only polynomial time.

### Stationary distribution
This is just $p(Y_1=h_1)$. We set this to $1/n$, i.e. uniform over $h=1,\dots,n$.

### Emission distribution
Given that the target haplotype is currently copying off of haplotype $j\in\{1\,\dots,n\}$, the emission probability is

$$p(H_{n+1,\ell}=H_{h_i,\ell} \mid {Y_i = h_i}) = 1-e^{-\theta/n}.$$

(Note this is a bit different than what they used in their original paper.)

### Transition distribution

Given that we are currently copying from haplotype $h_\ell$, what is the probability of $Y_{\ell+1}$? With probability $e^{-\rho/n}$, no recombination happens. Or, with probability $(1-e^{-\rho/n})(1/n)$, a recombination happens but we recombine onto the same haplotype as before. Otherwise, we are equally probable to recombine onto all other haplotypes:

$$p(Y_{\ell+1}=h_{\ell+1}\mid Y_\ell=h_\ell) = \begin{cases}
e^{-\rho/n} + (1-e^{-\rho/n})/n,&h_{\ell+1}=h_\ell\\
(1-e^{-\rho/n})/n,&\text{otherwise.}
\end{cases}
$$

Note that in matrix form, where $T_{ij} = p(Y_{\ell+1}=j\mid Y_\ell=i)$, we have 

$$T=e^{-\rho/n}I + \frac{1-e^{-\rho/n}}{n}\mathbf{1}\mathbf{1^T},$$

i.e. diagonal plus rank-one. This will be useful soon, as it will lead to an order-of-magnitude speedup in the standard forward backward algorithm.

## The forward-backward algorithm

Above we saw that naively evaluating the likelihood of an HMM would require exponential time. However, by exploiting the Markov property and using dynamic programmming, we can reduce this to polynomial time.

(I adopt the notation of Bishop (section 13.2) here.)

Define the *forward* quantity 

$$\hat{\alpha}(z_n) = p(z_n\mid x_1,\dots,x_n),$$ 

i.e. the conditional probability of the hidden state $z_n$ given the first $n$ observations. It's easy to see that the following recursion holds:

\begin{align}
\hat{\alpha}(z_{n+1}) =\frac{ \sum_{z_n}  p(x_{n+1}\mid{z_{n+1}})p(z_{n+1}\mid z_n)\hat{\alpha}(z_n)}
{p(x_{n+1} \mid x_1,\dots,x_n)}
\end{align}

Thus, if we recursively compute

$$\hat{\alpha}(z_{n+1}) \propto p(x_{n+1}\mid z_{n+1}) \sum_{z_n}  p(z_{n+1}\mid z_n)\hat{\alpha}(z_n)$$

and store the normalizing constants $c_1\,\dots,c_N$, then 

$$\mathbb{P}(x_1,\dots,x_N) = \prod_{i=1}^N c_i.$$

## Posterior decoding

Suppose that we are interested in the posterior distribution 

$$p(z_n \mid x_1,\dots,x_N) = \hat{\alpha}(z_n) \hat{\beta}(z_n)$$

where $$\hat{\beta}(z_n) = \frac{p(x_{n+1},\dots,x_N \mid z_n)}{p(x_{n+1},\dots,x_N\mid x_1,\dots,x_n)}.$$

$\hat{\beta}(z_n)$ can be computed by a similar recursion:

$$\hat{\beta}(z_n) = c^{-1}_{n+1}\sum_{z_{n+1}} \hat{\beta}(z_{n+1}) p(x_{n+1}\mid z_{n+1}) p(z_{n+1}\mid z_n),$$

where $c_{n+1}$ was computed during the forward pass.

## Implementing the L-S model
We will implemented the forward-backward algorithm and use it to obtain the posterior, which will give us a way to estimate ancestry.

In [38]:
import jax
import numpy as np
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp


def ls_fb(D, pos, rho, theta):
    "Compute posterior distribution in Li-Stephens algorithm"
    L, n = D.shape

    # emission probability
    e = jnp.array([jnp.exp(-theta / n), -jnp.expm1(-theta / n)], dtype=jnp.float64)

    alpha_hat_0 = (1.0 / n) * e[D[0]]
    c0 = alpha_hat_0.sum()
    alpha_hat_0 /= c0

    # transition quantity
    delta = jnp.diff(pos)
    T = jnp.exp(-rho * delta / n)

    def fwd(alpha_hat_i, x):
        t, d = x
        # alpha_hat[i+1] = e[d] * (alpha_hat[i] @ T)
        #                = e[d] * (alpha_hat[i] @ (diag + const * 11^T))
        #                = e[d] * (alpha_hat[i] * d + const), because |alpha_hat[i]| = 1
        alpha_hat_i1 = e[d] * (t * alpha_hat_i + (1. - t) / n)
        c = alpha_hat_i1.sum()
        alpha_hat_i1 /= c
        return alpha_hat_i1, (alpha_hat_i1, c)

    _, (alpha_hat, c) = jax.lax.scan(fwd, alpha_hat_0, (T, D[1:]))
    alpha_hat = jnp.concatenate([alpha_hat_0[None], alpha_hat])
    c = jnp.concatenate([c0[None], c])

    def bwd(beta_hat_i, x):
        ci, t, d = x
        beta_hat = e[d] * (t * beta_hat_i + (1. - t) / n * beta_hat_i.sum()) / ci
        return beta_hat, beta_hat

    beta_hat_L = jnp.ones_like(alpha_hat_0)
    _, beta_hat = jax.lax.scan(bwd, beta_hat_L, (c[1:], T, D[1:]), reverse=True)
    beta_hat = jnp.concatenate([beta_hat, beta_hat_L[None]])
    return alpha_hat * beta_hat, c

In [39]:
import msprime as msp

sim = msp.simulate(
    sample_size=8, mutation_rate=1e-8, recombination_rate=1e-8, length=1e5, Ne=1e4
)

In [40]:
X = sim.genotype_matrix()
pos = np.array([v.position for v in sim.variants()])
D = abs(X[:3, 1:] - X[:3, :1])
a, c = ls_fb(D, pos[:3], 1e-4, 1e-4)

In [41]:
a.sum(1)

DeviceArray([0.99536138, 1.00166592, 1.        ], dtype=float64)

In [16]:
D

array([[1, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 1],
       [0, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 1],
       [1, 1, 1, 1, 0, 1, 1]], dtype=int8)